In [1]:
# Import Modules
import pandas as pd
import os
import json
import numpy as np
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import yfinance as yf
from pathlib import Path
import sqlalchemy as sql
%matplotlib inline

In [2]:
#1. download historical data 1 ~ 2 symbolsCorrelations...
#2. Build Calculation Lib to caclculate TR, SR, PER, CORR, ....
#3. Create the matrix with result
# Symbol | Moving average | Total Return | Sharpe Ratio | PER price to earings | Corelation to SPY |..... * 660 symbols
#4. Store back to database table - "PERFORMANCE_MATRIX"

# function takes the history data frame as input and SYMBOL LIST NEEDS TO BE A VARIABLE FOR THIS CODE TO WORK
def stock_analzer(HistoryDF,symbol_list):
    performancematrix=pd.DataFrame(columns=['Symbol','Moving Average','Total Return','Sharpe Ratio','Average Covariance to spy','Average Beta'])
    for stock in symbol_list :
        #get ticker name
        ticker=stock
        #load closing prices
        closes=HistoryDF.loc[:,[(stock, 'close')]]
        closes=closes.iloc[:,0]
        #find dailyreturns
        dailyreturn=closes.pct_change().dropna()
        # total return
        totalreturn=closes[-1]/closes[0]
        # moving average
        mavgs=closes.rolling(window=60).mean().dropna()
        mavgs=mavgs.sum()/mavgs.count()
        #average annual retrun
        avg_anual_return=dailyreturn.mean()*252
        #annualized standard deviation using 252 as number of trading days in a year
        avg_std=perchange.std(axis = 0, skipna = True)*np.sqrt(252)    
        #sharpe ratios
        sharpe=avg_anual_return/avg_std
        #covariance to spy
            #getting closing values
        spy=historical_data_fetcher('SPY').iloc[:,3]
            #daily returns on spy
        spydr=spy.pct_change().dropna()
            #spy variance
        spyvar=spydr.rolling(window=60).var().dropna()
            #covariance with stock and spy
        cov=dailyreturn.rolling(window=60).cov(spydr)    
            #beta values
        beta=cov/spyvar


        row=pd.DataFrame( {'Symbol': ticker , 'Moving Average': mavgs,'Total Return':totalreturn,'Sharpe Ratio':sharpe,'Average Covariance to spy':cov.mean(),'Average Beta':beta.mean()}, index=[0])
        performancematrix=performancematrix.append(row,ignore_index = True)
    return performancematrix